In [2]:
import urllib
import os
import pandas as pd

In [4]:
province = ["http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/arezzo.pdf",
"http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/firenze.pdf",
"http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/grosseto.pdf",
"http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/livorno.pdf",
"http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/lucca.pdf",
"http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/massa.pdf",
"http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/pisa.pdf",
"http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/pistoia.pdf",
"http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/prato.pdf",
"http://www.arpat.toscana.it/temi-ambientali/amianto/mappatura/edifici/siena.pdf"]

In [12]:
def scarica_pdf(link):
    urllib.urlretrieve(link, os.path.join(os.getcwd(), "PDF", os.path.basename(link)))
    

In [13]:
for provincia in province:
    scarica_pdf(provincia)

<h2> proviamo a effettuare la trasformazione del pdf in txt attraverso PDFminer </h2>

In [5]:
from cStringIO import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = file(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text 

In [70]:
#convert(os.path.basename(province[0])).split("\n\n")

<h2> esito negativo, utilizziamo TABULA </h2>

In [71]:
#os.listdir(os.getcwd())

<h2> aggreghiamo i dati estratti con tabula </h2>

In [13]:
toscana = pd.DataFrame(columns = [u'provincia', u'comune', u'Ragione sociale', u'indirizzo', u'civico', u'destinazione', u'sup_coperta'])

In [14]:
for item in ['tabula-arezzo.csv',
 'tabula-firenze.csv',
 'tabula-grosseto.csv',
 'tabula-livorno.csv',
 'tabula-lucca.csv',
 'tabula-massa.csv',
 'tabula-pisa.csv',
 'tabula-pistoia.csv',
 'tabula-prato.csv',
 'tabula-siena.csv']:
    url = os.path.join(os.getcwd(), "Tabula", item)
    toscana = pd.concat([toscana, pd.read_csv(url, encoding='utf-8', sep=",")])

<h2> rimuoviamo le intestazioni di colonna in eccesso </h2>

In [15]:
toscana = toscana[toscana["provincia"] != "provincia"]

In [16]:
#si risolve una stranezza unicode di importazione tabula
for columns in range(0,toscana.shape[1]):
    for rows in range(0,toscana.shape[0]):
        try: 
            unicode(toscana.iloc[rows, columns])
        except:
            for caracter in toscana.iloc[rows, columns]:
                try:
                    unicode(caracter)
                except:
                    toscana.iloc[rows, columns] = toscana.iloc[rows, columns].replace(caracter, "X")

<h2> rimuoviamo le interruzioni di linea all'interno delle celle create da Tabula </h2>

In [17]:
def replace(x):
    if type(x) == str or type(x) == unicode:
        x = x.replace('\r', '')
    else:
        x = x[0].replace('\r', '')
    return x
    
toscana["indirizzo"] = toscana["indirizzo"].map(lambda x: x.replace('"', ''))
toscana["indirizzo"] = toscana["indirizzo"].map(lambda x: replace(x))

toscana["Ragione sociale"] = toscana["Ragione sociale"].map(lambda x: x.replace('"', ''))
toscana["Ragione sociale"] = toscana["Ragione sociale"].map(lambda x: replace(x))


<h2> prima prova di geolocalizzazione </h2>

In [18]:
import geopy

In [19]:
from geopy.geocoders import Nominatim
geolocator =  Nominatim()
location = geolocator.geocode("Firenze Via di Novoli 23")
print(location.address)

42/23, Via di Novoli, Careggi, Firenze, FI, TOS, 50127, Italia


In [20]:
location.latitude, location.longitude

(43.793649, 11.2202068)

In [30]:
def coordinate(via):
    try:
        geolocator = Nominatim()
        location = geolocator.geocode(via, addressdetails=True)
        print location.address
        return location.latitude, location.longitude
    except:
        pass
    

In [37]:
print coordinate("PIAZZA RESISTENZA BIBBIENA")

Piazza Resistenza, Bibbiena, Case Fraternità, Bibbiena, AR, TOS, Italia
(43.6933234, 11.8188218)


In [23]:
toscana["geo"] = toscana["indirizzo"] + " , " + toscana["comune"] +  " , " + "Italia"

In [24]:
toscana["geo"][0:100]#.map(lambda x: coordinate(x))

0                         DEL PANTANO , AREZZO , Italia
1                       VIA CHIASSAIA , AREZZO , Italia
2              PIAZZA POGGIO DEL SOLE , AREZZO , Italia
3                          VIA ROMANA , AREZZO , Italia
4                          PATRIGNONE , AREZZO , Italia
5                   PIAZZA DEL POPOLO , AREZZO , Italia
6                         P.TA FAENZI , AREZZO , Italia
7                 PIAZZA GUALCHIERE , BIBBIENA , Italia
8                 PIAZZA RESISTENZA , BIBBIENA , Italia
9          SS.71 LOCALITA' PIANACCI , BIBBIENA , Italia
10                     VIA XXV APRILE , BUCINE , Italia
11              VIA DANTE ALIGHIERI , CAPOLONA , Italia
12              VIA FIRENZE , CASTEL FOCOGNANO , Italia
13         PONTE DEL RIO , CASTEL SAN NICCOLO' , Italia
14         PONTE DEL RIO , CASTEL SAN NICCOLO' , Italia
15         VIA ARETINA , CASTELFRANCO DI SOPRA , Italia
16          VIALE EUROPA , CASTIGLION FIBOCCHI , Italia
17            MANCIANO , CASTIGLION FIORENTINO ,

In [46]:
toscana["coordinate"] = toscana["geo"].map(lambda x: coordinate(x))

In [65]:
def estrai_lat_long(x, posizione):
    try:
        return x[posizione]
    except:
        pass


toscana["latitudine"] = toscana["coordinate"].map(lambda x: estrai_lat_long(x,0))

In [66]:
toscana["longitudine"] = toscana["coordinate"].map(lambda x: estrai_lat_long(x,1))

In [74]:
toscana.to_excel("toscana.xlsx")

In [69]:
toscana.to_csv("toscana.csv", encoding = "utf-8")

In [5]:
toscana = pd.read_excel("toscana.xlsx")

In [8]:
os.chdir("..")
os.chdir("..")

data_folder = os.path.join(os.getcwd(), "dati", "MinAmbiente", "PNA_W")
data_folder_arpat = os.path.join(os.getcwd(), "dati", "RegioneToscana")

In [9]:
dati_PNA_W = pd.read_csv(os.path.join(data_folder, 'Toscana_2013.csv'), sep = ";")
dati_Arpat =  pd.read_csv(os.path.join(data_folder_arpat,  "toscana.csv"), sep=",")

In [23]:
confronto = pd.concat([dati_Arpat["provincia"].value_counts(), dati_PNA_W["Provincia"].value_counts()], axis = 1)

In [25]:
confronto.columns = ["ARPAT", "MINISTERO"]

<h2> Se i dati del ministero sono completi ed aggiornati al 2013 e quelli della Toscana sono completi ed aggiornati al 2007 possiamo sapere quanti interventi di bonifica sono stati effettuati per provincia</h2>

In [27]:
confronto["differenza"] = confronto["ARPAT"] - confronto["MINISTERO"]

In [37]:
confronto["var"] = confronto["MINISTERO"] / confronto["ARPAT"] * 100 - 100

In [38]:
confronto

,ARPAT,MINISTERO,differenza,var
FI,378,342,36,-9.523810
PI,171,149,22,-12.865497
GR,111,110,1,-0.900901
LI,104,98,6,-5.769231
LU,103,96,7,-6.796117
SI,80,74,6,-7.500000
MS,69,68,1,-1.449275
AR,59,59,0,0.000000
PT,52,47,5,-9.615385
PO,18,18,0,0.000000
